# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [2]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [16]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)

soup = BeautifulSoup(resp.text)
movies = soup.find(id="sbox_mid").find_all("option")
for i in movies[1:]:
    print("ID:",i["value"]," Movie:",i["data-name"])

ID: 10836  Movie: 魔鬼剋星2(1989)
ID: 10828  Movie: 阿紫
ID: 10824  Movie: 雙面追緝
ID: 10820  Movie: 愛回來的那7天
ID: 10819  Movie: 屍人莊殺人事件
ID: 10818  Movie: 我們仍未知道那天所看見的花名 劇場版
ID: 10807  Movie: ID4星際終結者
ID: 10806  Movie: 北海小英雄
ID: 10805  Movie: 守靈魘
ID: 10790  Movie: 失眠的北風吹來愛情
ID: 10750  Movie: 派遣死神事件簿
ID: 10531  Movie: 好萊塢殺人事件：曼森血案
ID: 10245  Movie: 日間演奏會散場時
ID: 10838  Movie: 鋼鐵雨：深潛行動
ID: 10712  Movie: 超危險駕駛
ID: 10825  Movie: 《新天堂樂園》30週年數位修復版
ID: 10822  Movie: 淒厲人僧前傳
ID: 10815  Movie: 盧貝松之聖女貞德： 數位修復版
ID: 10808  Movie: 野雀之詩
ID: 10802  Movie: 曼哈頓戀舞曲
ID: 10801  Movie: 詭絲
ID: 10800  Movie: 咒冥曲
ID: 10797  Movie: 戀夏時光
ID: 10793  Movie: 星之彩
ID: 10792  Movie: 屍人診所
ID: 10771  Movie: 廢宅人生
ID: 10770  Movie: 盧米埃：光與影的故事
ID: 10753  Movie: 致命窺弒
ID: 10730  Movie: 魔王的女兒
ID: 10711  Movie: 拂曉
ID: 7703  Movie: 解凍屍篇
ID: 10788  Movie: 翻供
ID: 10810  Movie: 小路與我的秘密驛站
ID: 10789  Movie: 不丹是教室
ID: 10775  Movie: 藍色恐懼：數位修復版
ID: 10686  Movie: 綠光 經典數位修復
ID: 10608  Movie: 屍速列車：感染半島
ID: 5350  Movie: 打噴嚏
ID: 10779  Movie: 里斯本的故事 25

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [21]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10057

In [24]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'avi=eyJpdiI6InAyYTdQTUZqaG8xUE5pVEtKZ0dJbGc9PSIsInZhbHVlIjoiMEs5ZUI1UWdHeUJPWEx5R0dKNzNqdz09IiwibWFjIjoiNDlhY2M4YjZhNzQ5ZDBmYjA4N2NiN2Q3MDUwZGRjMTY2M2YzMDUwNTM2NzkxNzMzNWI4ZDM5NGM0NzA3YjMyNCJ9; BX=91d4bjhfihuuf&b=3&s=f2; GUC=AQEBAQFfKk1fMkId7AS-; A1=d=AQABBM_7KF8CEKuHwg3NJSdtCaJRR5yLtJAFEgEBAQFNKl8yXwAAAAAA_SMAAAcIz_soX5yLtJA&S=AQAAAssaTUZgoR7NA83ymdflhPw; A3=d=AQABBM_7KF8CEKuHwg3NJSdtCaJRR5yLtJAFEgEBAQFNKl8yXwAAAAAA_SMAAAcIz_soX5yLtJA&S=AQAAAssaTUZgoR7NA83ymdflhPw; A1S=d=AQABBM_7KF8CEKuHwg3NJSdtCaJRR5yLtJAFEgEBAQFNKl8yXwAAAAAA_SMAAAcIz_soX5yLtJA&S=AQAAAssaTUZgoR7NA83ymdflhPw&j=WORLD; rxx=121u1639coz.20n915ux&v=1; _ga=GA1.4.410887265.1596521421; _gid=GA1.4.1888236141.1596521421; nexagesuid=eea348c95fe540da854eb6f67e1e5aa3; __gads=ID=180e906a5ab1246d:T=1596521427:S=ALNI_MbntHQJov1pEKZYcFECmppbHHVM2A; cmp=t=1596525598&j=0; browsed_movie=eyJpdiI6Ing2czh0Nlk5U1orVlwvakptcUtDbnBnPT0iLCJ2YWx1ZSI6IlZCekI3R2hERlZQZ2VLcm9ONklwZGhnRUhNXC9uc0xQTTRlNnFWa0pYaW5iRDRkT2JZNzNlQm1hYmZpeFpneXk5dzZEeUMyK01hOENMdjFYZm13TkxUZ0ltTWpkc255RFB5S0xnM1JDUDJ2ejNsUzNvNWg1VUZjSUVyV2pueGlMUCIsIm1hYyI6IjgxZTUxOGU3ZWEzMDQ5YzBmNDgyMTlhZjNlNjUwYTU3MWVjNmRlODBlMWZhYWY3YTU2MWM4NDVjY2RjYTlmOWEifQ%3D%3D; XSRF-TOKEN=eyJpdiI6IndzclEwT2JSVHMrN2lqNklwdXA1N0E9PSIsInZhbHVlIjoiaXRBak8yV0dLdThhM2crbHUrc3lSQmdlVnNJWHRFc1JXOVwveUVJaGRTY1VyYkU1TXp5b05vN3VxbTZxT2hGUkIxTDkzUGh5K05xWVJ5azdUeVNQTk9BPT0iLCJtYWMiOiJhYWUyMDhjZTgyOTE3NWIxZmZkNTMxYTdlZGE0MDY3ZmJjYjlkZDZhYTBjOWI1NTEwYjRlMzM2ZjJhNmQwNTkzIn0%3D; ms55=eyJpdiI6IjdJZDVzXC8yYys4bnN2ZUI1ckVjZzN3PT0iLCJ2YWx1ZSI6IjBsbGNUSHAzN2trUE1SREh4bzlzNE5LdDVONzBzY0l4d3dWTHY0ZXI4UXBJWmh1SmdrZjk0d1ZramRzSlMyXC8zVlZHRk9sczc4NzZ5Zk8xVlwvOHFvV3c9PSIsIm1hYyI6IjI4YTgxOTAzYmY5NTA4ZWNkYzdiZTlmOWM2NjUwZTFhMDViZDQ0OGQzZmE4ZmJmNmMxOTM5NGYzZGEwZTk5ZGUifQ%3D%3D; nexagesd=8',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
"""for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))"""
resp.json()

[{'area_id': 3, 'title': '台北二輪', 'priority': 149, 'ts_update': 1547194527},
 {'area_id': 2, 'title': '台中', 'priority': 125, 'ts_update': 1547507368}]

### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [26]:
# 指定放映地區
area_id = 3

In [35]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find("ul",class_="date_timetable_picker").find_all("input")

# 列印播放日期
for date in movie_date:
    if date.get("value") is not None:
        print(date["value"])

2020-08-04


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [38]:
# 選定要觀看的日期
date = "2020-08-04"

In [42]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼
soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})
html

[<ul class="area_time _c jq_area_time" data-theater_name="景美佳佳戲院" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=17" data-theater_url="" id="theater_id_17">
 <li class="adds">
 <a href="https://movies.yahoo.com.tw/theater_result.html/id=17">景美佳佳戲院</a>
 <span>02-29330333</span>
 </li>
 <li class="taps">
 <span class="tapR">數位</span> </li>
 <li class="time _c">
 <div class="input_picker jq_input_picker">
 <input class="gabtn" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', 'AI終結戰']" data-movie_date="08.04" data-movie_time="13:00" data-movie_title="AI終結戰" data-movie_type="數位" data-news="news" id="73899548" name="schedule_list" type="radio" value="2020-08-04 13:00:00"/>
 <label class="" for="73899548">13:00</label>
 <input class="gabtn" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', 'AI終結戰']" data-movie_date="08.04" data-movie_time="17:00" data-movie_title="AI終結戰" data-movie_type="數位" data-news="news" id="73899549" name="schedule_list" type="radio" value="2020-08-04 17:00:00"/>
 <label 

In [45]:
for i in html:
    print("----------------------------------------------------------------------")
    print("電影: ",i["data-theater_name"])
    print("放映類型: ",i.find(class_="tapR").text)
    for j in i.find_all("input"):
        print(j["value"])

----------------------------------------------------------------------
電影:  景美佳佳戲院
放映類型:  數位
2020-08-04 13:00:00
2020-08-04 17:00:00
2020-08-04 21:00:00
